# FrozenLake-v0

The agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile.

The episode ends when you reach the goal or fall in a hole. You receive a reward of 1 if you reach the goal, and zero otherwise.




## Q-learning

Q learning is a type of value-based RL algorithm. Say we had a situation like the one below. We have a robot, that has to go till the end, avoiding the obstacles. Lets say hitting any obstacle gives you a negative reward of -10. Lets say getting to the end gives you a +50 reward. Each step gives you a negative reward of -1, because the goal is to get to the end as quick as possible. 

### The Q-table

A Q-table is just a table that maps out the maximum expected future reward, for each action at each state. Going back to the previous example, a state would just be a block, and an action would basically be which direction you choose to move in from that block. There are 4 possible actions at each state, moving left, right, down, or up.

The Q-table is basically a cheat-sheet for the agent that tells it which action is the best action to take from each state. The Q-table itself improves with each iteration of the game. 


### Q-learning Algorithm 

The Q function has 2 inputs, the state and the action and based on this it computes the maximum expected future reward. Here is the equation for it:

Q-learning can be implemented as follows:

Q(s,a)+=α⋅[r+γ⋅maxαQ(s′)−Q(s,a)]
s: is the previous state

a: is the previous action

Q(): is the Q-learning algorithm

s’: is the current state

alpha: is the learning rate, set generally between 0 and 1. Setting the alpha value to 0 means that the Q-values are never updated, thereby nothing is learned. If we set the alpha to a high value such as 0.9, it means that the learning can occur quickly.

gamma: It is the discount factor that is set between 0 and 1. This model the fact that future rewards are worth less than immediate rewards.

max: is the maximum reward that is attainable in the state following the current one (the reward for taking the optimal action thereafter).

The algorithm can be interpreted as:

Initialize the Q-values table, Q(s, a).

Observe the current state, s.

Take an action, a, for that state based on the selection policy.

Pick that action, and observe the reward, r, as well as the new state, s’.

Now update the Q-value for the state with the help of the observed reward and the maximum reward possible for the next state.

Place the state to the new state, and repeat the process until a terminal state is reached.

Thus, alpha is the learning rate. If the reward or transition function is random, then alpha should change over the period, approaching zero at infinity. This has to effect approximating the expected outcome of an inner product (T(transition)*R(reward)), when one of the two, or both, has random behavior.

Whereas, gamma is the value of future rewards. It can change the learning quite a bit and can be a dynamic or static value. If it is equal to one, the agent values future reward JUST AS MUCH as a current reward. This means, in ten actions, if an agent does something good this is JUST AS VALUABLE as doing this action directly. So learning doesn't work well at high gamma values.

Similarly, a gamma of zero will cause the agent to only value immediate rewards, which only works with very detailed reward functions.

### Import Dependencies

- Numpy for Qtable 
- OpenAI Gym for Taxi Environment
- Random to generate random numbers

First Install Gym and other required packages to be able to run Toy Text environments

In [1]:
import numpy as np
import gym
import random

In [2]:
env = gym.make("FrozenLake-v0")
env.render()


SFFF
FHFH
FFFH
HFFG


In [3]:
action_size = env.action_space.n
print('Action Size - ',action_size)

state_size = env.observation_space.n
print('State Size - ', state_size)

Action Size -  4
State Size -  16


In [4]:
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [9]:
total_episodes = 5000        # Total episodes
total_test_episodes = 100     # Total test episodes
max_steps = 99                # Max steps per episode

learning_rate = 0.7           # Learning rate
gamma = 0.8                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

In [10]:

rewards = []
avg_epsilon = []

print('Q-Learning -------------------------------')
# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0

    for step in range(max_steps):

        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0,1)

        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * 
                                    np.max(qtable[new_state, :]) - qtable[state, action])

        total_rewards += reward
        # Our new state is state
        state = new_state

        # If done : finish episode
        if done == True: 
            break
    
        if(step == max_steps-1):
        #print('Max Step Reached for Episode - ', episode)
        #print('Epsilon value at Max Step - ', epsilon)
            avg_epsilon.append(epsilon)

    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    rewards.append(total_rewards)

    
print("Number of Training Episodes - " + str(total_episodes))
print ("Score over time: " +  str(sum(rewards)/total_episodes))
print("Average Epsilon value Per Episode: " + str(sum(avg_epsilon)/len(avg_epsilon)))
print(qtable)
print(" ")


Q-Learning -------------------------------
Number of Training Episodes - 5000
Score over time: 0.3226
Average Epsilon value Per Episode: 0.010407262485706588
[[2.04733962e-03 8.74070167e-03 1.54221627e-04 1.09148711e-03]
 [9.26225724e-05 1.49412737e-03 1.09279990e-04 1.33454134e-02]
 [1.33635779e-01 1.00479008e-04 1.02779526e-04 1.01600590e-04]
 [9.93827764e-05 2.99994008e-05 4.04300930e-05 8.78764877e-05]
 [1.63879576e-02 1.18034387e-03 2.19957370e-04 1.14203648e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.18185696e-06 5.28868620e-06 6.87695340e-02 6.46897262e-06]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.65298711e-03 2.83853794e-05 2.42035240e-03 5.54826035e-02]
 [1.22880132e-03 3.15920520e-01 1.00691144e-02 7.68806814e-03]
 [3.79656360e-03 6.31149039e-01 6.96259485e-03 3.98669071e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.67440978e-03 1.9013

In [11]:
env.reset()
rewards = []
avg_steps = []

print('Testing ---------------------------------')
for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    #print("****************************************************")
    #print("EPISODE ", episode)

    for step in range(max_steps):

        # UNCOMMENT IT IF YOU WANT TO SEE OUR AGENT PLAYING
        #env.render()
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])

        new_state, reward, done, info = env.step(action)

        total_rewards += reward

        if done:
            #env.render()
            print ("Episode - "+ str(episode) + ",  Score - ", total_rewards)
            #avg_steps.append(step)
            break
        state = new_state
    avg_steps.append(step)
    rewards.append(total_rewards)
            

env.close()

print("Learning Rate value - " + str(learning_rate))
print("Number of Episodes - " + str(total_test_episodes))
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))
print("Average num of Steps Per Episode: " + str(sum(avg_steps)/total_test_episodes))


Testing ---------------------------------
Episode - 0,  Score -  0.0
Episode - 1,  Score -  0.0
Episode - 2,  Score -  0.0
Episode - 3,  Score -  0.0
Episode - 4,  Score -  0.0
Episode - 5,  Score -  0.0
Episode - 6,  Score -  0.0
Episode - 7,  Score -  0.0
Episode - 8,  Score -  1.0
Episode - 9,  Score -  1.0
Episode - 10,  Score -  0.0
Episode - 11,  Score -  0.0
Episode - 12,  Score -  0.0
Episode - 13,  Score -  0.0
Episode - 14,  Score -  1.0
Episode - 15,  Score -  1.0
Episode - 16,  Score -  0.0
Episode - 17,  Score -  0.0
Episode - 18,  Score -  0.0
Episode - 19,  Score -  1.0
Episode - 20,  Score -  1.0
Episode - 21,  Score -  0.0
Episode - 22,  Score -  1.0
Episode - 23,  Score -  0.0
Episode - 24,  Score -  0.0
Episode - 25,  Score -  0.0
Episode - 26,  Score -  1.0
Episode - 27,  Score -  1.0
Episode - 28,  Score -  1.0
Episode - 29,  Score -  0.0
Episode - 30,  Score -  0.0
Episode - 31,  Score -  0.0
Episode - 32,  Score -  1.0
Episode - 33,  Score -  0.0
Episode - 34,  S

## Score Over Time for baseline Model

Training - 0.3226

Testing - 0.37

#### Questions- 

##### 1. Establish a baseline performance. How well did your RL Q-learning do on your problem?

The Q-learning did perform decently with the baseline parameters.
I got a Training Score(Average rewards) of 0.32 and Testing 0.37. 
The average number of steps per episode is 27.88


##### 2. What are the states, the actions and the size of the Q-table?

SFFF       (S: starting point, safe)
FHFH       (F: frozen surface, safe)
FFFH       (H: hole, fall to your doom)
HFFG       (G: goal, where the frisbee is located)

Actions are - Up, down, right, left

Size of Q table - (state_size, action_size) (16,4)

##### 3. What are the rewards? Why did you choose them?

The default reward policy is - 
1 for each success
0 for failure

We will try a different policy too after tuning the hyperparameters


##### 4. How did you choose alpha and gamma in the following equation?

The alpha & gamma are default values taken from the baseline parameters provided. I will choose these later while tuning. 


##### 5. Try at least one additional value for alpha and gamma. How did it change the baseline performance?
Done in Tuning Sheet

##### 6. Try a policy other than maxQ(s', a'). How did it change the baseline performance?
Done in Tuning Sheet

##### 7. How did you choose your decay rate and starting epsilon? Try at least one additional value for epsilonand the decay rate. How did it change the baseline performance? 
Done in Tuning Sheet

##### 8. What is the value of epsilon when if you reach the max steps per episode?
0.010407262485706588

##### 9. What is the average number of steps taken per episode?
27.88 steps per episode 

##### 10. Does Q-learning use value-based or policy-based iteration?
Q-learning uses value-based iteration as it uses a policy to update the Q values. It is a off-policy. 

##### 11. What is meant by expected lifetime value in the Bellman equation?
Expected Lifetime value in bellman equation is the Total Expected Reward for the Agent in its lifetime which consists of the max of Future reward discounted in time. The lifetime of the agent is one Episode. 